In [5]:
from IPython.display import HTML
shell = get_ipython()

def adjust_font_size():
    display(HTML('''<style>
        body{
            font-size: 32px;
        }
    '''))

if adjust_font_size not in shell.events.callbacks['pre_execute']:
    shell.events.register('pre_execute', adjust_font_size)

In [6]:
def printmatrix(m,pad = 4): 
    for r in m:
        for d in r:
            print(f"{str(d):>{pad}}", end = " ")
        print()

In [7]:
seq1 = "TTAGCCAGT"
seq2 = "TAAGACATTTTAC"
gappenalty = -8
scoringmatrix = [[00 for i in range(0,len(seq1)+1)] for j in range(0,len(seq2)+1)]
directionmatrix = [["." for i in range(0,len(seq1)+1)] for j in range(0,len(seq2)+1)]
print(scoringmatrix)
scoringmatrix[0][1] = 77
print(scoringmatrix)
printmatrix(scoringmatrix)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 77, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
   0   77    0    0    0    0    0    0    0    0 
   0    0    0    0    0    0    0    0    0    0 

In [8]:
for i in range(0,len(scoringmatrix[0])):
    scoringmatrix[0][i] =  i*gappenalty
    directionmatrix[0][i] = "h "
for i in range(0,len(scoringmatrix)):
    scoringmatrix[i][0] = i*gappenalty
    directionmatrix[i][0]  = "v "

for r in range(1,len(scoringmatrix)):
    for c in range(1,len(scoringmatrix[r])):
        vert = scoringmatrix[r-1][c] + gappenalty
        horz = scoringmatrix[r][c-1] + gappenalty
        diag = scoringmatrix[r-1][c-1]
        if(seq1[c-1] == seq2[r-1]):
            diag += 10
        else:
            diag -= 10
        scoringmatrix[r][c] = max(vert,horz,diag)
        #directionmatrix[r][c] = "v or h or d"
        if diag >= horz and diag >= vert:
            directionmatrix[r][c] = "\u2196"
        if horz > diag and horz > vert:
            directionmatrix[r][c] = "\u2190"
        if vert > diag and vert > horz:
            directionmatrix[r][c] = "\u2191"

printmatrix(scoringmatrix,4)
print(seq1)
print(seq2)


   0   -8  -16  -24  -32  -40  -48  -56  -64  -72 
  -8   10    2   -6  -14  -22  -30  -38  -46  -54 
 -16    2    0   12    4   -4  -12  -20  -28  -36 
 -24   -6   -8   10    2   -6  -14   -2  -10  -18 
 -32  -14  -16    2   20   12    4   -4    8    0 
 -40  -22  -24   -6   12   10    2   14    6   -2 
 -48  -30  -32  -14    4   22   20   12    4   -4 
 -56  -38  -40  -22   -4   14   12   30   22   14 
 -64  -46  -28  -30  -12    6    4   22   20   32 
 -72  -54  -36  -38  -20   -2   -4   14   12   30 
 -80  -62  -44  -46  -28  -10  -12    6    4   22 
 -88  -70  -52  -54  -36  -18  -20   -2   -4   14 
 -96  -78  -60  -42  -44  -26  -28  -10  -12    6 
-104  -86  -68  -50  -52  -34  -16  -18  -20   -2 
TTAGCCAGT
TAAGACATTTTAC


In [9]:
--TG
CATT

NameError: name 'TG' is not defined

In [14]:
printmatrix(directionmatrix,0)

v  h  h  h  h  h  h  h  h  h  
v  ↖ ↖ ← ← ← ← ← ← ↖ 
v  ↑ ↖ ↖ ← ← ← ↖ ← ← 
v  ↑ ↖ ↖ ↖ ↖ ↖ ↖ ← ← 
v  ↑ ↖ ↑ ↖ ← ← ← ↖ ← 
v  ↑ ↖ ↖ ↑ ↖ ↖ ↖ ← ↖ 
v  ↑ ↖ ↑ ↑ ↖ ↖ ← ↖ ↖ 
v  ↑ ↖ ↖ ↑ ↑ ↖ ↖ ← ← 
v  ↖ ↖ ↑ ↑ ↑ ↖ ↑ ↖ ↖ 
v  ↖ ↖ ↖ ↑ ↑ ↖ ↑ ↖ ↖ 
v  ↖ ↖ ↖ ↑ ↑ ↖ ↑ ↖ ↖ 
v  ↖ ↖ ↖ ↑ ↑ ↖ ↑ ↖ ↖ 
v  ↑ ↑ ↖ ↑ ↑ ↖ ↖ ↖ ↑ 
v  ↑ ↑ ↑ ↖ ↖ ↖ ↑ ↖ ↑ 
